#### Compute dbscan .csv file for comparison

In [58]:
from typing import Dict, List, Tuple
from sklearn.cluster import DBSCAN
from collections import defaultdict
import pandas as pd
import json
import os
import numpy as np

dataset_name = 'test_ride_niek'
annotations_file = f"{dataset_name}.json"    # This file is the raw azure coco annotations file.
processed_detections_file = f"dbscan_{dataset_name}_processed_detections.json"
dbscan_csv_file = f"{dataset_name}_results_dbscan.csv"
ALLOWED_CATEGORIES = [3, 4, 5]

In [59]:
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0]);  yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2]);  yB = min(boxA[3], boxB[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    areaA = (boxA[2]-boxA[0])*(boxA[3]-boxA[1])
    areaB = (boxB[2]-boxB[0])*(boxB[3]-boxB[1])
    return inter / (areaA + areaB - inter) if (areaA+areaB-inter)>0 else 0

def attach_by_iou(df_db, ann_df):
    grouped = ann_df.groupby(['image_name','object_category'])
    new_aids = []

    for _, row in df_db.iterrows():
        key = (row.image_name, row.object_category)
        if key not in grouped.groups:
            new_aids.append(np.nan)
            continue
        
        cand = grouped.get_group(key)
        # compute IoU for each candidate
        ious = cand.apply(
            lambda r: iou(
                [row.x1, row.y1, row.x2, row.y2],
                [r.x1,    r.y1,    r.x2,    r.y2]
            ), axis=1
        )
        best = ious.idxmax()
        new_aids.append(cand.loc[best, 'id'])   # ann_df’s annotation ID column is 'id'

    df_db['annotation_id'] = new_aids
    return df_db

def load_data() -> Tuple[List[Dict], List[Dict], List[Dict]]:
    """Load and prepare annotation and detection data for evaluation.
    
    Loads the processed annotations and detections from JSON files, extracts the relevant
    data, and adds GPS information to both annotations and detections.
    
    Returns:
        Tuple[List[Dict], List[Dict], List[Dict]]: A tuple containing:
            - annotations: List of ground truth annotations with GPS data
            - detections: List of predicted detections with GPS data
            - images: List of image metadata from the detections file
    """
    with open(annotations_file, 'r') as f:
        ground_truth_annotations = json.load(f)
    with open(processed_detections_file, 'r') as f:
        predicted_detections = json.load(f)
    images = predicted_detections.get('images', [])
    detections = predicted_detections['annotations']
    annotations = ground_truth_annotations['annotations']
    
    # Add GPS data to both annotations and detections
    add_gps_to_annotations(annotations, images)
    add_gps_to_annotations(detections, images)
    return annotations, detections, images

def add_gps_to_annotations(annotations: List[Dict], images: List[Dict]) -> List[Dict]:
    """
    Augment each annotation with GPS data from its corresponding image.

    Args:
        annotations (List[Dict]): List of annotation dictionaries, each
                                  containing an 'image_id' field.
        images (List[Dict]): List of image entries (e.g., COCO images)
                              each with 'id' and optional 'gps_data'.

    Returns:
        List[Dict]: The same list of annotations, modified in place, where
                    each annotation has a new 'gps_data' field set to the
                    matching image's GPS info or None if unavailable.
    """
    # Build a mapping from image_id to gps_data
    image_id_to_gps = {img['id']: img.get('gps_data') for img in images}
    for ann in annotations:
        ann['gps_data'] = image_id_to_gps.get(ann['image_id'])
    return annotations

def filter_detections_to_annotated_images(annotations: List[Dict], 
                                          detections: List[Dict]) -> Tuple[List[Dict], List[Dict]]:
    """Filter detections to only keep those from images that have ground truth annotations.
    
    Args:
        annotations (List[Dict]): List of ground truth annotations
        detections (List[Dict]): List of predicted detections
    
    Returns:
        Tuple[List[Dict], List[Dict]]: A tuple containing:
            - annotations: The original annotations (unchanged)
            - filtered_detections: Only detections from images that have ground truth annotations
    """
    annotated_image_ids = set(ann['image_id'] for ann in annotations)
    filtered_detections = [det for det in detections if det['image_id'] in annotated_image_ids]
    return annotations, filtered_detections

def get_base_filename(filename: str) -> str:
    """
    Extract the base filename from a path or URL, stripping query parameters.

    Args:
        filename (str): Full file path or URL potentially containing query strings.

    Returns:
        str: The base filename without any directory path or query suffix.
    """
    base = os.path.basename(filename)
    return base.split('?')[0]

def create_filename_to_id_mapping(coco_data: Dict) -> Dict[str, int]:
    """
    Build a lookup mapping base filenames to COCO image IDs.

    Args:
        coco_data (Dict): COCO dataset dictionary containing an 'images'
                          list of image entries with 'file_name' and 'id'.

    Returns:
        Dict[str, int]: Dictionary mapping each base filename (stripped of path
                        and query) to its corresponding image ID.
    """
    return {get_base_filename(img['file_name']): img['id'] 
            for img in coco_data.get('images', [])}
    
def group_annotations_by_image(annotations: Dict) -> Dict[int, List[Dict]]:
    """
    Organize a flat list of annotations into groups keyed by image ID.

    Args:
        annotations (Dict): Dictionary with an 'annotations' key containing
                            a list of annotation dicts, each having an 'image_id'.

    Returns:
        Dict[int, List[Dict]]: Mapping from image_id to a list of annotations
                               belonging to that image.
    """
    annotations_by_image = {}
    for ann in annotations['annotations']:
        img_id = ann['image_id']
        if img_id not in annotations_by_image:
            annotations_by_image[img_id] = []
        annotations_by_image[img_id].append(ann)
    return annotations_by_image
    
def align_coco_ids(detections_coco: str, annotations_coco: str, 
                    output_file: str) -> Tuple[Dict, Dict]:
    """
    Align image and annotation IDs between two COCO-format JSON files based on matching filenames.

    Args:
        detections_coco (str): Path to the COCO-format file containing detection data.
        annotations_coco (str): Path to the COCO-format file containing ground truth annotations.
        output_file (str): Path to the output file where the updated annotations will be saved.

    Returns:
        Tuple[Dict, Dict]: A tuple containing the original detections dictionary and the updated
                           annotations dictionary with aligned IDs and merged metadata.
    """
    print("Aligning IDs between COCO files...")
    
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    with open(detections_coco, 'r') as f:
        detections = json.load(f)
    with open(annotations_coco, 'r') as f:
        annotations = json.load(f)
    
    # Create mappings
    detections_filename_to_id = create_filename_to_id_mapping(detections)
    annotations_filename_to_id = create_filename_to_id_mapping(annotations)
    
    # Create ID mapping
    id_mapping = {annotations_filename_to_id[filename]: det_id
                    for filename, det_id in detections_filename_to_id.items()
                    if filename in annotations_filename_to_id}
    
    # Create detections data mapping
    detections_data = {img['id']: {
        'gps_data': img['gps_data'],
        'record_timestamp': img['record_timestamp']
    } for img in detections['images']}
    
    # Update annotations
    for img in annotations['images']:
        base_filename = get_base_filename(img['file_name'])
        if base_filename in detections_filename_to_id:
            new_id = detections_filename_to_id[base_filename]
            img['id'] = new_id
            if new_id in detections_data:
                img['gps_data'] = detections_data[new_id]['gps_data']
                img['record_timestamp'] = detections_data[new_id]['record_timestamp']
                img['file_name'] = base_filename
    
    # Update annotation IDs
    annotations_by_image = group_annotations_by_image(annotations)
    new_annotation_id = 1
    
    for old_image_id, new_image_id in id_mapping.items():
        if old_image_id in annotations_by_image:
            for ann in sorted(annotations_by_image[old_image_id], key=lambda x: x['id']):
                ann['id'] = new_annotation_id
                ann['image_id'] = new_image_id
                new_annotation_id += 1
    
    # Sort and save
    annotations['images'] = sorted(annotations['images'], key=lambda x: x['id'])
    annotations['annotations'] = sorted(annotations['annotations'], key=lambda x: x['id'])
    
    with open(output_file, 'w') as f:
        json.dump(annotations, f, indent=2)
    
    print(f"Matched {len(id_mapping)} images")
    print(f"Total annotations: {len(annotations['annotations'])}")

def cluster_images_by_gps_and_select_per_class(annotations: List[Dict], 
                                               detections: List[Dict], 
                                               eps_meters: float = 20, 
                                               min_samples: int = 1) -> Tuple[List[Dict], List[Dict]]:
    """Cluster images by GPS location and select one image per cluster per class.
    
    Clusters images by GPS location using DBSCAN. For each cluster and each class,
    keeps only one image (the one with the most detections of that class).
    Filters annotations and detections to only those in the selected images.
    
    Args:
        annotations (List[Dict]): List of ground truth annotations (with gps_data)
        detections (List[Dict]): List of predicted detections (with gps_data)
        eps_meters (float): DBSCAN epsilon in meters. Default is 20.
        min_samples (int): DBSCAN min_samples. Default is 1.
    
    Returns:
        Tuple[List[Dict], List[Dict]]: A tuple containing:
            - filtered_annotations: Annotations from selected images
            - filtered_detections: Detections from selected images
    """
    # Build image_id to GPS mapping from annotations
    image_gps = {}
    for ann in annotations:
        img_id = ann['image_id']
        if img_id not in image_gps and ann.get('gps_data'):
            gps = ann['gps_data']
            if 'latitude' in gps and 'longitude' in gps:
                image_gps[img_id] = (gps['latitude'], gps['longitude'])

    image_ids = list(image_gps.keys())
    if not image_ids:
        return annotations, detections

    coords = np.array([image_gps[img_id] for img_id in image_ids])
    coords_rad = np.radians(coords)
    db = DBSCAN(eps=eps_meters/6371008.8, min_samples=min_samples, metric='haversine')
    labels = db.fit_predict(coords_rad)

    # Map image_id to cluster label
    image_id_to_cluster = {img_id: label for img_id, label in zip(image_ids, labels)}

    # For each cluster and class, keep one image (with most detections of that class)
    cluster_class_to_images = defaultdict(lambda: defaultdict(list))
    image_class_count = defaultdict(lambda: defaultdict(int))
    for det in detections:
        img_id = det['image_id']
        class_id = det['category_id']
        if img_id in image_id_to_cluster:
            cluster = image_id_to_cluster[img_id]
            cluster_class_to_images[cluster][class_id].append(img_id)
            image_class_count[img_id][class_id] += 1

    selected_image_ids = set()
    for cluster, class_to_images in cluster_class_to_images.items():
        for class_id, img_ids in class_to_images.items():
            best_img_id = max(img_ids, key=lambda img_id: image_class_count[img_id][class_id])
            selected_image_ids.add(best_img_id)

    filtered_annotations = [ann for ann in annotations if ann['image_id'] in selected_image_ids]
    filtered_detections = [det for det in detections if det['image_id'] in selected_image_ids]
    return filtered_annotations, filtered_detections

In [60]:
#align_coco_ids(processed_detections_file, annotations_file, processed_annotations_file)

In [61]:
all_annotations, all_detections, all_images = load_data()
annotations, detections = filter_detections_to_annotated_images(all_annotations, all_detections)

print(f'Loaded {len(annotations)} annotations and {len(detections)} detections from {len(all_images)} images')
print(f'Unique # of images: {len(set(ann["image_id"] for ann in annotations))}')
print(f'Unique # of images in predictions: {len(set(det["image_id"] for det in detections))}')
# cluster & select
filtered_anns, filtered_dets = cluster_images_by_gps_and_select_per_class(
    annotations=annotations,
    detections=detections,
    eps_meters=5
)

# build a DataFrame in the same format as trackers' CSVs
df_db = pd.DataFrame(filtered_dets)
ann_df = pd.DataFrame(filtered_anns)

id2fn = { img['id']: os.path.basename(img['file_name']) for img in all_images }
df_db['image_name']  = df_db['image_id'].map(id2fn)
df_db = df_db.rename(columns={'category_id':'object_category'})

ann_df['image_name'] = ann_df['image_id'].map(id2fn)
ann_df = ann_df.rename(columns={'category_id':'object_category'})

df_db  = df_db [df_db['object_category'].isin(ALLOWED_CATEGORIES)]
ann_df = ann_df[ann_df['object_category'].isin(ALLOWED_CATEGORIES)]

for D in (df_db, ann_df):
    # the `.bbox` column is a list [x,y,w,h]; stack into a 2D array
    arr = np.vstack(D['bbox'].values)
    D['x1'], D['y1'], D['w'], D['h'] = arr.T
    D['x2'] = D['x1'] + D['w']
    D['y2'] = D['y1'] + D['h']
    
print(f'Unique # of images: {len(df_db["image_name"].unique())}')

df_db = attach_by_iou(df_db, ann_df)

print(f'# of non-matched detections: {df_db["annotation_id"].isnull().sum()}')
print(f'# of matched detections: {len(df_db)}')

df_db = df_db[df_db['annotation_id'].notnull()]
df_db['annotation_id'] = df_db['annotation_id'].astype(int)
df_out = df_db[['image_name','id','object_category','annotation_id']].copy()
df_out.columns = ['image_name','ID','object_category','annotation_id']
df_out.to_csv(dbscan_csv_file, index=False)
print(f"Wrote DBSCAN selection to {dbscan_csv_file}, retained {len(df_db)} rows")

Loaded 955 annotations and 737 detections from 2444 images
Unique # of images: 373
Unique # of images in predictions: 324
Unique # of images: 33
# of non-matched detections: 13
# of matched detections: 41
Wrote DBSCAN selection to test_ride_niek_results_dbscan.csv, retained 28 rows


### Compare Tracking Algorithms

In [62]:
import pandas as pd
import json
import os
import numpy as np

#dataset_name = 'recording_2025-05-14'
dataset_name = 'test_ride_niek'

GT_CSV    = f'{dataset_name}_reviewed.csv'
PRED_CSV  = f'{dataset_name}_tracks_ma30_bytetrack.csv'
ANNOT_JSON= f'{dataset_name}.json'
DATA_DIR  = dataset_name
ALLOWED_CATEGORIES = [3, 4, 5]  # Categories to consider for evaluation

In [65]:
def load_ground_truth(gt_csv):
    '''
    Load manual annotations and return a DataFrame with columns:
    ['image_name','gt_id','object_category'].
    Adjusts object_category from 0-4 to 1-5 to match predictions/JSON.
    '''
    df = pd.read_csv(gt_csv)
    df = df.rename(columns={'ID': 'gt_id'})
    # Shift object_category by +1 to align 0-4 → 1-5
    df['object_category'] = df['object_category'] + 1
    return df[['image_name', 'gt_id', 'object_category']]

def load_predictions(pred_csv):
    """
    Load tracker outputs and return a DataFrame with columns:
    ['image_name','trk_id','object_category','annotation_id'].
    Strips any directory prefix by taking basename.
    """
    df = pd.read_csv(pred_csv)
    df = df.rename(columns={'ID':'trk_id'})
    df['image_name'] = df['image_name'].apply(os.path.basename)
    return df[['image_name','trk_id','object_category','annotation_id']]

def assign_frame_indices(gt_df, pr_df):
    '''
    Create a global mapping from image_name to a consecutive frame index,
    based on sorted unique image_name values across GT and predictions.
    '''
    unique_names = sorted(set(gt_df['image_name']).union(pr_df['image_name']))
    name_to_frame = {name: idx+1 for idx, name in enumerate(unique_names)}
    # Apply mapping
    gt_df['frame'] = gt_df['image_name'].map(name_to_frame)
    pr_df['frame'] = pr_df['image_name'].map(name_to_frame)
    return gt_df, pr_df

def load_coco_bboxes(ann_json_path, allowed_categories=None):
    """
    Load COCO annotations JSON and return a DataFrame with columns:
    ['image_name','x1','y1','x2','y2','category_id','annotation_id'].
    Converts normalized bboxes to pixel coords using image width/height.
    Strips any directory prefix by taking basename of file_name.
    """
    coco = json.load(open(ann_json_path, 'r'))
    img_meta = {img['id']: img for img in coco['images']}
    records = []
    for ann in coco['annotations']:
        cid = ann['category_id']
        if allowed_categories and cid not in allowed_categories:
            continue
        img = img_meta.get(ann['image_id'])
        if img is None:
            continue
        w_img, h_img = img['width'], img['height']
        x, y, bw, bh = ann['bbox']  # normalized [0-1]
        px, py = int(x * w_img), int(y * h_img)
        pw, ph = int(bw * w_img), int(bh * h_img)
        fn = os.path.basename(img['file_name'])
        records.append({
            'image_name': fn,
            'x1': px, 'y1': py,
            'x2': px + pw, 'y2': py + ph,
            'category_id': cid,
            'annotation_id': ann['id']
        })
    return pd.DataFrame(records)

def attach_bboxes(df, ann_df, key_cols=('image_name','object_category')):
    """
    For each row in df, find all matching ann_df rows on key_cols,
    pick one at random (if any), and attach its bbox + annotation_id.
    If no match, fill with NaN.
    
    df must have columns matching key_cols ('object_category' in df 
    corresponds to 'category_id' in ann_df).
    ann_df must have ['image_name','category_id','x1','y1','x2','y2','annotation_id'].
    """
    # rename for uniformity
    ann = ann_df.rename(columns={'category_id': 'object_category'})
    # build groups
    groups = ann.groupby(list(key_cols))

    # preallocate columns
    out = df.copy()
    out[['x1','y1','x2','y2','annotation_id']] = np.nan

    # for reproducibility
    rng = np.random.default_rng(seed=42)

    # for each unique key tuple, sample once and broadcast
    for keys, subidx in out.groupby(list(key_cols)).groups.items():
        # keys is a tuple (image_name, object_category)
        # get candidate bboxes
        try:
            candidates = groups.get_group(keys)
        except KeyError:
            # no JSON bbox for this key
            continue
        # choose one row at random
        chosen = candidates.sample(n=1, random_state=rng).iloc[0]
        # assign to all matching df rows
        mask = (out['image_name'] == keys[0]) & (out['object_category'] == keys[1])
        out.loc[mask, ['x1','y1','x2','y2','annotation_id']] = \
            chosen[['x1','y1','x2','y2','annotation_id']].values

    return out

def object_level_retention(gt_df, pred_csv):
    import pandas as pd
    import os

    # load predictions
    pr = pd.read_csv(pred_csv)
    pr['image_name'] = pr['image_name'].apply(os.path.basename)
    print(f'Printing unique image names number in predictions: {len(pr["image_name"].unique())}')

    # build a unique mapping from (image_name, annotation_id) -> gt_id
    mapping = (
        gt_df[['image_name','annotation_id','gt_id']]
        .dropna(subset=['annotation_id'])              # drop any NaNs
        .astype({'annotation_id':'int'})                # ensure int dtype
        .drop_duplicates(subset=['image_name','annotation_id'])
    )

    merged = pr.merge(
        mapping,
        on=['image_name','annotation_id'],
        how='left'
    )

    # count how many distinct gt_id got recovered
    recovered = set(merged['gt_id'].dropna().astype(int))
    total_gt = gt_df['gt_id'].nunique()
    return len(recovered), total_gt

In [66]:
for dataset in ['test_ride_niek']:
    # Load and prepare data
    GT_CSV = f"{dataset}_reviewed.csv"
    ANNOT_JSON = f"{dataset}.json"
    gt_df = load_ground_truth(GT_CSV)
    ann_df = load_coco_bboxes(ANNOT_JSON, allowed_categories=ALLOWED_CATEGORIES)
    gt_df = attach_bboxes(gt_df, ann_df.rename(columns={'category_id':'object_category'}), 
                            key_cols=('image_name','object_category'))
    # Amount of unique objects in GT csv
    unique_gt_ids = gt_df['gt_id'].nunique()
    print(f"{dataset} - Unique GT objects: {unique_gt_ids}")
    
    for name, path in [
        ("ByteTrack",  f"{dataset}_tracks_ma30_bytetrack.csv"),
        ("DeepSORT",   f"{dataset}_tracks_ma30_cd40_ni3_deepsort.csv"),
        ("DBSCAN",     f"{dataset}_results_dbscan.csv"),
    ]:
        kept, total = object_level_retention(gt_df, path)
        print(f"{dataset} - {name:8s}:  {kept}/{total} objects recovered ({kept/total:.0%})")

test_ride_niek - Unique GT objects: 48
Printing unique image names number in predictions: 216
test_ride_niek - ByteTrack:  39/48 objects recovered (81%)
Printing unique image names number in predictions: 141
test_ride_niek - DeepSORT:  28/48 objects recovered (58%)
Printing unique image names number in predictions: 23
test_ride_niek - DBSCAN  :  0/48 objects recovered (0%)


#### Debugging potential mismatch

In [54]:
import pandas as pd

# Load the two lists of image names
db = pd.read_csv("test_ride_niek_results_dbscan.csv")
db_images = set(db["image_name"].unique())

# Print annotation_id of an image in both db_images and gt_images
image_filename = 'detection_20250514_121945667.jpg'
print(f"Annotation IDs in DBSCAN for 'image_name' {image_filename}:")
print(db[db['image_name'] == f'{image_filename}'])
print(f"Annotation IDs in GT for 'image_name' {image_filename}':")
print(gt_df[gt_df['image_name'] == f'{image_filename}'])

print(f"Images in DBSCAN output: {len(db_images)}")
gt_images = set(gt_df['image_name'].unique())
print(f"Images in GT CSV:         {len(gt_images)}")
print(f"Intersection:             {len(db_images & gt_images)}")
print(f'Intersected images: {sorted(db_images & gt_images)}')
print("Images in DBSCAN but not in GT:")
print(sorted(db_images - gt_images))

Annotation IDs in DBSCAN for 'image_name' detection_20250514_121945667.jpg:
                          image_name   ID  object_category  annotation_id
20  detection_20250514_121945667.jpg  498                3            728
Annotation IDs in GT for 'image_name' detection_20250514_121945667.jpg':
                          image_name  gt_id  object_category     x1     y1  \
23  detection_20250514_121945667.jpg      2                3  454.0  394.0   

       x2     y2  annotation_id  
23  482.0  431.0          431.0  
Images in DBSCAN output: 23
Images in GT CSV:         373
Intersection:             10
Intersected images: ['detection_20250514_121805969.jpg', 'detection_20250514_121810541.jpg', 'detection_20250514_121811053.jpg', 'detection_20250514_121811586.jpg', 'detection_20250514_121812135.jpg', 'detection_20250514_121945667.jpg', 'detection_20250514_121949317.jpg', 'detection_20250514_121950834.jpg', 'detection_20250514_121951354.jpg', 'detection_20250514_121952373.jpg']
Images in 

In [34]:
import json
import pandas as pd

# 1) Load your DBSCAN output
db = pd.read_csv("test_ride_niek_results_dbscan.csv")
db_ids = set(db['annotation_id'].astype(int))

# 2) Load the ground-truth COCO JSON and pull out its annotation IDs
with open("test_ride_niek.json", 'r') as f:
    coco = json.load(f)
gt_ids = {ann['id'] for ann in coco['annotations']}

# 3) Compare
print(f"DBSCAN IDs count: {len(db_ids)}")
print(f"GT JSON IDs count: {len(gt_ids)}")
print("DBSCAN IDs not in GT JSON:", sorted(db_ids - gt_ids))
print("Intersection size:", len(db_ids & gt_ids))

DBSCAN IDs count: 25
GT JSON IDs count: 955
DBSCAN IDs not in GT JSON: []
Intersection size: 25
